# Import required modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn import cross_validation
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns
import glob
import os
from sklearn.cross_validation import cross_val_score
plt.style.use('ggplot')

c:\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Define Functions

In [2]:
#A function to calculate the results
def result(x):
	conditions = [
	(x['GoalsFor'] > x['GoalsAgainst']),
	(x['GoalsFor'] == x['GoalsAgainst']),
	(x['GoalsFor'] < x['GoalsAgainst'])
	]
	choices = [1,0,-1]
	x['Result'] = np.select(conditions, choices)
	return x

In [3]:
def joined_result(x):
    if x == 'H':
        return 1
    elif x == 'D':
        return 0
    elif x == 'A':
        return -1

# Data Preperation

Read CSV Data

In [4]:
full_results=pd.read_csv(r'C:\Users\dbrown\Documents\Misc\Football\16_17\overall.csv')
fixtures = pd.read_csv(r'C:\Users\dbrown\Documents\Misc\Football\17_18\fixtures.csv')

Define base dataframes

In [5]:
full_stats = full_results[['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']]
fixtures = fixtures[['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']]
full_results_combo = full_stats.append(fixtures)
full_results_combo.drop_duplicates(subset=['Date','Div','HomeTeam','AwayTeam'],keep='last',inplace=True)
home_stats = full_results_combo.set_index(['Date','Div','HomeTeam'])
away_stats = full_results_combo.set_index(['Date','Div','AwayTeam'])

Work on dataframes

In [6]:
home_stats = home_stats[['FTHG','FTAG']]
home_stats.rename(columns={'FTHG':'GoalsFor','FTAG':'GoalsAgainst'},inplace='True')
home_stats['GoalDiff'] = home_stats['GoalsFor'] - home_stats['GoalsAgainst']
away_stats = away_stats[['FTAG','FTHG']]
away_stats.rename(columns={'FTAG':'GoalsFor','FTHG':'GoalsAgainst'},inplace='True')
away_stats['GoalDiff'] = away_stats['GoalsFor'] - away_stats['GoalsAgainst']
#Set dataframes with the result
home_stats = result(home_stats)
away_stats = result(away_stats)
#Rename columns in Home and Away stats
home_stats.rename(columns={'GoalsFor':'HomeGoalsFor','GoalsAgainst':'HomeGoalsAgainst','GoalDiff':'HomeGoalDiff','Result':'HomeResult'},inplace='True')
away_stats.rename(columns={'GoalsFor':'AwayGoalsFor','GoalsAgainst':'AwayGoalsAgainst','GoalDiff':'AwayGoalDiff','Result':'AwayResult'},inplace='True')

Set 2-6 game form values

In [7]:
for num in range(2,7):
        home = 'Home'
        away = 'Away'
        HomeGameForm="%s%sGameForm" % (home,num)
        HomeGameGoalsFor="%s%sGameGoalsFor" % (home,num)
        HomeGameGoalDiff="%s%sGameGoalDiff" % (home,num)
        AwayGameForm="%s%sGameForm" % (away,num)
        AwayGameGoalsFor="%s%sGameGoalsFor" % (away,num)
        AwayGameGoalDiff="%s%sGameGoalDiff" % (away,num)
        home_stats[HomeGameForm] = home_stats['HomeResult'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        home_stats[HomeGameGoalsFor] = home_stats['HomeGoalsFor'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        home_stats[HomeGameGoalDiff] = home_stats['HomeGoalDiff'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        away_stats[AwayGameForm] = away_stats['AwayResult'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        away_stats[AwayGameGoalsFor] = away_stats['AwayGoalsFor'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        away_stats[AwayGameGoalDiff] = away_stats['AwayGoalDiff'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))

Merge Home and Away Stats

In [8]:
home_noindex = home_stats.reset_index()
away_noindex = away_stats.reset_index()
home_merge = pd.merge(full_results_combo, home_noindex, on=['Date','Div','HomeTeam'])
home_merge.rename(columns={'HomeTeam_x':'HomeTeam','AwayTeam_x':'AwayTeam'},inplace=True)
away_merge = pd.merge(full_results_combo, away_noindex, on=['Date','Div','AwayTeam'])
away_merge.rename(columns={'HomeTeam_x':'HomeTeam','AwayTeam_x':'AwayTeam'},inplace=True)
home_away_merge = pd.merge(home_merge, away_merge, on=['Date','Div','AwayTeam'])
home_away_merge.rename(columns={'HomeTeam_x':'HomeTeam'},inplace=True)
home_away_merge = home_away_merge[['Date', 'Div','HomeTeam','AwayTeam','HomeGoalsFor', 'HomeGoalsAgainst', 'HomeGoalDiff', 'HomeResult','Home2GameForm', 'Home2GameGoalsFor', 'Home2GameGoalDiff','Home3GameForm', 'Home3GameGoalsFor', 'Home3GameGoalDiff',
       'Home4GameForm', 'Home4GameGoalsFor', 'Home4GameGoalDiff',
       'Home5GameForm', 'Home5GameGoalsFor', 'Home5GameGoalDiff',
       'Home6GameForm', 'Home6GameGoalsFor', 'Home6GameGoalDiff',
       'AwayGoalsFor', 'AwayGoalsAgainst',
       'AwayGoalDiff', 'AwayResult', 'Away2GameForm', 'Away2GameGoalsFor',
       'Away2GameGoalDiff', 'Away3GameForm', 'Away3GameGoalsFor',
       'Away3GameGoalDiff', 'Away4GameForm', 'Away4GameGoalsFor',
       'Away4GameGoalDiff', 'Away5GameForm', 'Away5GameGoalsFor',
       'Away5GameGoalDiff', 'Away6GameForm', 'Away6GameGoalsFor',
       'Away6GameGoalDiff']]

Set relevant columns and clean data

In [9]:
home_away_merge_idx = home_away_merge.set_index(['Date','Div','HomeTeam','AwayTeam'])
#home_away_merge_idx_relevant = home_away_merge_idx[['HomeGoalsFor','AwayGoalsFor','HomeResult','3GameFormDiff']]
#home_away_merge_cleaned = home_away_merge_idx_relevant.dropna()

Set 2-6 game form difference values

In [10]:
for num in range(2,7):
        home = 'Home'
        away = 'Away'
        HomeGameForm="%s%sGameForm" % (home,num)
        HomeGameGoalsFor="%s%sGameGoalsFor" % (home,num)
        HomeGameGoalDiff="%s%sGameGoalDiff" % (home,num)
        AwayGameForm="%s%sGameForm" % (away,num)
        AwayGameGoalsFor="%s%sGameGoalsFor" % (away,num)
        AwayGameGoalDiff="%s%sGameGoalDiff" % (away,num)
        GameFormDiff = "%sGameFormDiff" % (num)
        GameGoalsForDiff = "%sGameGoalsForDiff" % (num)
        GameGoalDiffDiff = "%sGameGoalDiffDiff" % (num)
        home_away_merge_idx[GameFormDiff] = home_away_merge_idx[HomeGameForm] - home_away_merge_idx[AwayGameForm]
        home_away_merge_idx[GameGoalsForDiff] = home_away_merge_idx[HomeGameGoalsFor] - home_away_merge_idx[AwayGameGoalsFor]
        home_away_merge_idx[GameGoalDiffDiff] = home_away_merge_idx[HomeGameGoalDiff] - home_away_merge_idx[AwayGameGoalDiff]

Deal with NaN values

In [11]:
home_away_merge_idx_zeroed = home_away_merge_idx.fillna(0)
home_away_merge_idx_dropped = home_away_merge_idx.dropna()
home_away_merge_idx_zeroed_reset = home_away_merge_idx_zeroed.reset_index()

Get the latest stats

In [12]:
home_stats_latest = home_away_merge_idx_zeroed_reset.groupby('HomeTeam').last()
home_stats_latest.reset_index(inplace=True)
home_stats_latest.set_index(['Date','Div','HomeTeam','AwayTeam'],inplace=True)
latest_fixtures = home_stats_latest[home_stats_latest["HomeResult"]==0]

In [13]:
latest_fixtures

HomeGoalsFor  \
Date     Div HomeTeam         AwayTeam                         
18/02/18 G1  AEK              Xanthi                     0.0   
17/02/18 E2  AFC Wimbledon    Bristol Rvs                0.0   
10/02/18 N1  AZ Alkmaar       VVV Venlo                  0.0   
17/02/18 SC2 Airdrie Utd      Queens Park                0.0   
16/02/18 F2  Ajaccio GFCO     Reims                      0.0   
18/02/18 SP1 Alaves           La Coruna                  0.0   
17/02/18 SC2 Albion Rvs       Raith Rvs                  0.0   
         SP2 Alcorcon         Tenerife                   0.0   
         EC  Aldershot        Macclesfield               0.0   
         F1  Amiens           Toulouse                   0.0   
04/02/18 B1  Anderlecht       Mechelen                   2.0   
17/02/18 F1  Angers           St Etienne                 0.0   
03/02/18 SC3 Annan Athletic   Cowdenbeath                1.0   
17/02/18 T1  Antalyaspor      Kayserispor                0.0   
         B1  Antwerp          Oostende                   0.0   
11/02/18 G1  Apollon          Levadeiakos                1.0   
17/02/18 SC2 Arbroath         Alloa                      0.0   
         G1  Asteras Tripolis Atromitos                  0.0   
18/02/18 I1  Atalanta         Fiorentina                 0.0   
09/02/18 SP1 Ath Bilbao       Las Palmas                 0.0   
18/02/18 SP1 Ath Madrid       Ath Bilbao                 0.0   
11/02/18 G1  Atromitos        Olympiakos                 2.0   
18/02/18 D1  Augsburg         Stuttgart                  0.0   
12/02/18 F2  Auxerre          Nimes                      0.0   
10/02/18 I2  Avellino         Cesena                     1.0   
17/02/18 P1  Aves             Maritimo                   0.0   
         SC2 Ayr              East Fife                  0.0   
11/02/18 SP1 Barcelona        Getafe                     0.0   
17/02/18 E3  Barnet           Accrington                 0.0   
10/02/18 E1  Barnsley         Sheffield Weds             1.0   
...                                                      ...   
16/02/18 B1  St Truiden       Anderlecht                 0.0   
17/02/18 SC3 Stenhousemuir    Edinburgh City             0.0   
         E3  Stevenage        Yeovil                     0.0   
         SC3 Stirling         Annan Athletic             0.0   
10/02/18 E0  Stoke            Brighton                   1.0   
17/02/18 E1  Sunderland       Brentford                  0.0   
         EC  Sutton           Wrexham                    0.0   
         E3  Swindon          Port Vale                  0.0   
19/02/18 P1  Tondela          Sp Lisbon                  0.0   
18/02/18 I1  Torino           Juventus                   0.0   
17/02/18 EC  Torquay          Dag and Red                0.0   
18/02/18 T1  Trabzonspor      Buyuksehyr                 0.0   
17/02/18 EC  Tranmere         Bromley                    0.0   
         F1  Troyes           Metz                       0.0   
18/02/18 N1  Twente           Sparta Rotterdam           0.0   
17/02/18 I1  Udinese          Roma                       0.0   
16/02/18 N1  VVV Venlo        Groningen                  0.0   
         F2  Valenciennes     Ajaccio                    0.0   
         SP2 Valladolid       Huesca                     0.0   
17/02/18 I2  Venezia          Avellino                   0.0   
             Virtus Entella   Pescara                    0.0   
         N1  Vitesse          Excelsior                  0.0   
11/02/18 B1  Waasland-Beveren Club Brugge                1.0   
17/02/18 B1  Waregem          Eupen                      0.0   
         D1  Wolfsburg        Bayern Munich              0.0   
10/02/18 EC  Wrexham          Halifax                    1.0   
         G1  Xanthi           Kerkyra                    1.0   
09/02/18 T1  Yeni Malatyaspor Kasimpasa                  1.0   
29/04/17 EC  York             Forest Green               2.0   
18/02/18 N1  Zwolle           Ajax                       0.0   

               

# Logistic Regression on HomeResult

In [14]:
y = home_away_merge_idx_dropped['HomeResult']
X = home_away_merge_idx_dropped[[
    '6GameFormDiff', '6GameGoalsForDiff', '6GameGoalDiffDiff']]
logreg = LogisticRegression()

In [15]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.8)
logreg.fit(X_train, y_train)
logreg.score(X_train, y_train)
cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean()

0.47645009915160674

In [16]:
y = home_away_merge_idx_dropped['HomeResult']
X = home_away_merge_idx_dropped[[
    '6GameFormDiff', '6GameGoalsForDiff', '6GameGoalDiffDiff']]

logreg = LogisticRegression()

#X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.9)
logreg.fit(X, y)
logreg.score(X, y)

y_pred = logreg.predict(X)
#metrics.accuracy_score(y_test, y_pred)

#cross_val_score(logreg, X, y, cv=10, scoring='accuracy')

#cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean()

#y_pred_prob = logreg.predict_proba(X_test)[:, -1]

#logreg.predict_proba(X_test)[:, -3]

#y_pred_prob

latest_fixtures_cut = latest_fixtures[[
    '6GameFormDiff', '6GameGoalsForDiff', '6GameGoalDiffDiff']]
probs = logreg.predict_proba(latest_fixtures_cut)
probs_df = pd.DataFrame(probs)
probs_df.rename(columns={0:'AwayWinProb',1:'DrawProb',2:'HomeWinProb'},inplace='True')
latest_fixtures_cut_noind = latest_fixtures_cut.reset_index()
pred_join = latest_fixtures_cut_noind.join(probs_df)
pred_join_cut = pred_join[["Date","Div","HomeTeam","AwayTeam","HomeWinProb"]]

In [18]:
pred_join_cut[pred_join_cut["HomeWinProb"]>0.65]

,Date,Div,HomeTeam,AwayTeam,HomeWinProb
5,18/02/18,SP1,Alaves,La Coruna,0.675833
32,17/02/18,P1,Benfica,Boavista,0.681948
95,18/02/18,N1,Feyenoord,Heracles,0.695649
162,16/02/18,F1,Monaco,Dijon,0.682071
173,16/02/18,F2,Nimes,Tours,0.689446
185,17/02/18,F1,Paris SG,Strasbourg,0.732428
190,17/02/18,SC3,Peterhead,Berwick,0.677059
